In [1]:
## La version de python utilisée est 3.12.7

!pip install -r requirements.txt -q

In [2]:
import pandas as pd
import requests
import lxml as lxml
from bs4 import BeautifulSoup
import io as io
import math
import gzip
import shutil
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import json
from pandasgui import show
import numpy as np
from io import BytesIO
from folium.plugins import HeatMap
import nbconvert
from concurrent.futures import ThreadPoolExecutor
import time

from script import request_tri
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [3]:
# Charger le fichier CSV contenant les transactions
transactions = pd.read_csv("data/base.csv",sep=";",encoding="utf-8")

C:\Users\flori\AppData\Local\Temp\ipykernel_52588\2978799986.py:2: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  transactions = pd.read_csv("data/base.csv",sep=";",encoding="utf-8")


In [4]:
# Fonction wrapper avec gestion des erreurs et temporisation
def check_inondable_parallel(args):
    latitude, longitude = args
    attempts = 2  # Nombre de tentatives en cas d'échec
    for _ in range(attempts):
        try:
            return request_tri.check_inondable(latitude, longitude)
        except requests.exceptions.RequestException as e:
            # print(f"Erreur de connexion pour la commune (lat: {latitude}, lon: {longitude}): {e}")
            time.sleep(2)  # Délai de 2 secondes avant de réessayer
    return "Erreur de connexion" 

# Préparer les données nécessaires (coordonnées)
coordinates = list(zip(transactions['latitude'], transactions['longitude']))

# Limiter le nombre de threads simultanés pour éviter la surcharge
with ThreadPoolExecutor(max_workers=5) as executor:  # Limite de 5 threads
    results = list(executor.map(check_inondable_parallel, coordinates))

# Convertir les résultats en DataFrame
results_df = pd.DataFrame(results, columns=['results', 'identifiant_tri', 'libelle_type_inondation', 'code_scenario'])

# Fusionner les résultats avec les données originales
transactions = pd.concat([transactions.reset_index(drop=True), results_df], axis=1)

In [5]:
# Sauvegarder le fichier avec les résultats
transactions = transactions.rename(columns={'results': 'zone_inondable'})
transactions['zone_inondable'] = transactions['zone_inondable'].replace({2: 0, 3: 0})
transactions.to_csv("data/transactions_with_zone_inondable.csv",sep=";",encoding="utf-8", index=False)

In [6]:
transactions = pd.read_csv("data/transactions_with_zone_inondable.csv",sep=";",encoding="utf-8")

C:\Users\flori\AppData\Local\Temp\ipykernel_52588\1473045797.py:1: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  transactions = pd.read_csv("data/transactions_with_zone_inondable.csv",sep=";",encoding="utf-8")


La requête de l'API Géorisques renseigne 4 informatons :
- la présence ou non dans une zone inondable (results)
- si présence, la zone géographique TRI (Territoire à Risque Important d'Inondation) à laquelle le point géographique est associé 
- si présence, le code scénario associé. Il existe 4 scénarios distincts :
    - l'aléa de forte probabilité (01For) dénommé "évènement fréquent" avec une période de retour de 10 à 30 ans
    - l'aléa de moyenne probabilité (02Moy) dénommé "évènement moyen", avec une période d eretour de 100 à 300 ans
    - l'aléa de moyenne probabilité avec changement climatique (03Mcc) dénommé "évènement moyen avec changement climatique" (qui est une majoration d'un évènement moyen)
    - l'aléa de faible probabilité (03Fai), dénommé "évènement extrême" avec une période de retour d'au moins 1000 ans

La période de retour est la durée moyenne au cours de laquelle un évènement d'une même intensité est amené à se reproduire

- si présence, l'aléa d'inondation associé : submersion marine, débordements des cours d'eau, ruissellement et débordements des eaux souterraines

In [7]:
# Étape 1 : Compter et afficher le nombre de transactions par commune
transaction_counts = transactions.groupby('nom_commune').size().sort_values(ascending=False)
print(transaction_counts)

nom_commune
Nice                 7721
Nantes               5048
Bordeaux             4600
Toulon               3069
Cannes               2831
                     ... 
San-Giuliano            1
Serra-di-Fiumorbo       1
Taglio-Isolaccio        1
Aregno                  1
Île-de-Batz             1
Length: 946, dtype: int64


In [8]:
# Filtrer les transactions pour les Maisons
filtered_maisons = transactions[transactions['type_local'] == 'Maison']
filtered_maisons = filtered_maisons.groupby('nom_commune').filter(
    lambda group: len(group) >= 10 and group['zone_inondable'].sum() > 5 and (group['zone_inondable'] == 0).any()
)

# Filtrer les transactions pour les Appartements
filtered_appartements = transactions[transactions['type_local'] == 'Appartement']
filtered_appartements = filtered_appartements.groupby('nom_commune').filter(
    lambda group: len(group) >= 10 and group['zone_inondable'].sum() > 5 and (group['zone_inondable'] == 0).any()
)

# Combiner les deux sous-ensembles filtrés
filtered_transactions = pd.concat([filtered_maisons, filtered_appartements])


In [9]:
# Étape 3 : Calculer la moyenne agrégée des prix par commune et par zone inondable
filtered_transactions['prix_m2'] = filtered_transactions['valeur_fonciere'] / filtered_transactions['surface_reelle_bati']

In [10]:
# Moyenne par commune et zone inondable
mean_prices = filtered_transactions.groupby(['nom_commune', 'zone_inondable'])['prix_m2'].mean().unstack()

In [11]:
def process_data(filtered_data, output_file):
    # Calcul du prix moyen par commune et par zone inondable
    mean_by_zone = (
        filtered_data
        .groupby(['nom_commune', 'zone_inondable'])['prix_m2']
        .mean()
        .unstack(fill_value=0)  # Remplace NaN par 0
    )
    
    # Calcul du prix moyen total par commune
    mean_total = (
        filtered_data
        .groupby('nom_commune')['prix_m2']
        .mean()
    )
    
    # Renommer les colonnes pour les prix moyens par zone
    mean_by_zone = mean_by_zone.rename(columns={0: 'prix_moyen_non_inondable', 1: 'prix_moyen_inondable'})
    
    # Calcul de la différence relative en %
    mean_by_zone['écart'] = (
        (mean_by_zone['prix_moyen_inondable'] - mean_by_zone['prix_moyen_non_inondable']) / mean_by_zone['prix_moyen_non_inondable'] * 100
    )
    
    # Ajouter le nombre total de transactions par commune (nb_transactions)
    nb_transactions = filtered_data.groupby('nom_commune').size()
    
    # Ajouter le nombre de transactions inondables par commune
    nb_inondable = filtered_data[filtered_data['zone_inondable'] == 1].groupby('nom_commune').size()
    
    # Calculer la proportion de transactions inondables par commune (part_inondable)
    part_inondable = nb_inondable / nb_transactions * 100
    
    # Joindre les nouvelles colonnes au DataFrame 'mean_by_zone'
    mean_by_zone['nb_transactions'] = nb_transactions
    mean_by_zone['part_inondable'] = part_inondable
    
    # Ajouter la population par commune
    population_data = filtered_data.groupby('nom_commune')['Population'].max()
    mean_by_zone = mean_by_zone.join(population_data, on='nom_commune')
    
    # Trier les communes par population décroissante et sélectionner les 20 plus peuplées
    result_table = mean_by_zone.sort_values(by='Population', ascending=False).head(20)
    
    # Exporter le DataFrame final en CSV
    result_table.to_csv(output_file, encoding='utf-8-sig', sep=";", decimal=",", index=True)
    
    # Ajouter la colonne 'nom_commune' comme première colonne
    result_table = result_table.reset_index()

    # Réorganiser les colonnes pour mettre 'Population' en deuxième position
    cols = ['nom_commune', 'Population', 'prix_moyen_non_inondable', 'prix_moyen_inondable', 
            'écart', 'nb_transactions', 'part_inondable']
    result_table = result_table[cols]

    # Formatage des nombres avec séparateurs de milliers (espaces)
    result_table['prix_moyen_non_inondable'] = result_table['prix_moyen_non_inondable'].apply(lambda x: f"{x:,.0f}".replace(',', ' '))
    result_table['prix_moyen_inondable'] = result_table['prix_moyen_inondable'].apply(lambda x: f"{x:,.0f}".replace(',', ' '))
    result_table['écart'] = result_table['écart'].apply(lambda x: f"{x:,.1f}%".replace(',', ' '))
    result_table['nb_transactions'] = result_table['nb_transactions'].apply(lambda x: f"{x:,.0f}".replace(',', ' '))
    result_table['part_inondable'] = result_table['part_inondable'].apply(lambda x: f"{x:,.1f}%".replace(',', ' '))
    result_table['Population'] = result_table['Population'].apply(lambda x: f"{x:,.0f}".replace(',', ' '))


    # Retourner le tableau stylé
    return (
        result_table
        .style
        .hide(axis="index")  # Supprime l'index
        .set_caption(f"Top 20 des communes les plus peuplées - {output_file.split('/')[-1]}")
    )

# Processus pour les appartements
print("Calcul pour les appartements")
appart_table = process_data(filtered_appartements, 'data/moyenne_appartements.csv')
display(appart_table)  # Affiche dans le notebook

# Processus pour les maisons
print("\nCalcul pour les maisons")
maison_table = process_data(filtered_maisons, 'data/moyenne_maisons.csv')
display(maison_table)  # Affiche dans le notebook


Calcul pour les appartements


nom_commune,Population,prix_moyen_non_inondable,prix_moyen_inondable,écart,nb_transactions,part_inondable
Nice,348 085,5 030,4 433,-11.9%,7 476,13.0%
Nantes,323 204,3 705,3 911,5.6%,4 101,11.7%
Bordeaux,261 804,4 932,3 925,-20.4%,3 509,8.3%
Toulon,180 452,3 035,2 456,-19.1%,2 644,22.2%
Perpignan,119 656,1 626,1 597,-1.8%,1 895,25.3%
Rouen,114 083,2 811,2 616,-6.9%,1 932,18.1%
Caen,108 200,2 917,3 270,12.1%,1 730,2.3%
Dunkerque,86 788,2 151,1 992,-7.4%,495,12.3%
Béziers,80 341,1 612,1 334,-17.2%,1 018,2.7%
Cherbourg-en-Cotentin,77 808,2 169,2 331,7.4%,357,20.2%



Calcul pour les maisons


nom_commune,Population,prix_moyen_non_inondable,prix_moyen_inondable,écart,nb_transactions,part_inondable
Nice,348 085,7 189,16 753,133.0%,245,4.9%
Nantes,323 204,4 770,4 932,3.4%,947,2.3%
Bordeaux,261 804,5 696,5 208,-8.6%,1 091,3.7%
Toulon,180 452,4 810,4 091,-14.9%,425,10.8%
Perpignan,119 656,2 326,2 075,-10.8%,547,18.3%
Rouen,114 083,3 078,2 904,-5.7%,364,6.9%
Dunkerque,86 788,1 987,2 215,11.5%,607,11.2%
Béziers,80 341,2 476,2 416,-2.4%,473,4.0%
Cherbourg-en-Cotentin,77 808,2 402,2 389,-0.6%,567,10.6%
Saint-Nazaire,72 057,3 239,2 406,-25.7%,557,8.8%
